In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

# !pip install missingno
import missingno as msno
from datetime import date
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report, plot_roc_curve
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)


****1. Downloading the data****

In [ ]:
#submission_csv = pd.read_csv('../input/2022gdz22-datathon-case-2/submission.csv')
test_ = pd.read_csv('../input/gdz22-datathon-case-2-v3/2022gdz22-datathon-case-2/test.csv')
test = test_.copy()
train_ = pd.read_csv('../input/gdz22-datathon-case-2-v3/2022gdz22-datathon-case-2/train.csv')
train = train_.copy()

****2. EDA****

In [ ]:
print('train obs. points:',train.shape[0])
print('test obs points:', test.shape[0])
test.shape[0]/(train.shape[0]+test.shape[0])

Test set accounts for 13.7% of whole data.

In [ ]:
train['BAŞLAMA_TARİHİ_VE_ZAMANI'] = pd.to_datetime(train['BAŞLAMA_TARİHİ_VE_ZAMANI'], dayfirst = True)
train.set_index('BAŞLAMA_TARİHİ_VE_ZAMANI').resample('D')['cagri_count'].sum().plot()

In [ ]:
train = train[train['BAŞLAMA_TARİHİ_VE_ZAMANI']< '2022-03-24 23:58:01' ]

In [ ]:
train['BAŞLAMA_TARİHİ_VE_ZAMANI'].max()

In [ ]:
train['BAŞLAMA_TARİHİ_VE_ZAMANI'].min()

In [ ]:
# Column that existing in train but not in test.
[col for col in train.columns if col not in test.columns]
# cagri_count is the target column.

In [ ]:
# Column that is existing in test but not in train.
[col for col in test.columns if col not in train.columns]

In [ ]:

# Quick glance at a dataset
def check_df(data, x=5):
    print('################################# shape ##########################')
    print(data.shape )
    print('################################# type ##########################')
    print(data.dtypes)
    print('################################# head ##########################')
    #print(data.head(x))
    print('################################# tail ##########################')
    #print(data.tail(x))
    print('################################# null ##########################')
    print(data.isnull().sum().to_frame(name = 'counts').query('counts>0').sort_values(by = 'counts',ascending=False))
    print('################################# quantiles #####################')
    print(data.describe([0, 0.05, 0.5, 0.95, 0.99, 1]).T)
check_df(train)

In [ ]:


check_df(test)

It seems that we have got some outliers in 'KESINTI_SURESI', 'KENTSEL_OG', 'KENTSEL_AG', 'KENTALTI_AG'... since the mean and the median of those values are not similar. Also, some of the values does not fit in normal distribution.

In [ ]:
# Concatination of the test and the train set
df = pd.concat([test.assign(ind="test"), train.assign(ind="train")])

In [ ]:
def grab_col_names (dataframe, categorical = 10, cardinal =20):

    # categoricals
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtype == 'O']
    #  [col for col in df.columns if str(df[col].dtypes) in ["category","object","bool"]]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].dtype == 'O' and
                   dataframe[col].nunique() > cardinal]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].dtype != 'O' and
                   dataframe[col].nunique() < categorical]
    cat_cols += num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # numericals

    num_cols = [col for col in dataframe.columns if dataframe[col].dtype != 'O' and
                col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car


cat_cols, num_cols, cat_but_car = grab_col_names (df, categorical = 10, cardinal =20)

We have 8 Categorical Columns and 16 Numerical columns. From categoricals, 5 of them has high cardinality.

In [ ]:
cat_cols

In [ ]:
num_cols

In [ ]:
df.head()

In [ ]:
# Numerical Variable Analysis
def num_summary(dataframe, numeric_col, plot=False):
    print('#####################', numeric_col,'############################')
    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    print(dataframe[numeric_col].describe(quantiles).T)

    if plot:
        dataframe[numeric_col].hist()
        plt.xlabel(numeric_col)
        plt.title(numeric_col)
        plt.show(block=True)

for col in num_cols:
    num_summary(df, col, plot=True)

In [ ]:
for col in num_cols:
    sns.boxplot(x=df[col])
    plt.show()

In [ ]:
def cat_summary( dataframe, col_name, plot = False):
    print('#####################',col_name,'############################')
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                       'Ratio':100 * dataframe[col_name].value_counts()/len(dataframe)}))

    if plot:
        sns.countplot(x = dataframe[col_name], data = dataframe)
        plt.show(block = True)

for col in cat_cols:
    cat_summary(df, col, plot = True)

In [ ]:
# Target Summary with Categorical Columns:
def target_summary_with_cat(dataframe, target, categorical_col):
    print(pd.DataFrame({"TARGET_MEAN": dataframe.groupby(categorical_col)[target].mean()}), end="\n\n\n")

In [ ]:
for col in cat_cols:
    target_summary_with_cat(df, 'cagri_count', col)

In [ ]:
target_summary_with_cat(df, 'cagri_count', 'KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA')

In [ ]:
target_summary_with_cat(df, 'cagri_count', 'İLÇE')

In [ ]:
def target_summary_with_num(dataframe, target, numerical_col):
    print(dataframe.groupby(target).agg({numerical_col: 'mean'}))


#for col in num_cols:
    #target_summary_with_num(df, 'cagri_count', col)

In [ ]:
# Bağımlı değişkenin incelenmesi
df["cagri_count"].hist(bins=2)
plt.show()

# Bağımlı değişkenin logaritmasının incelenmesi
np.log1p(df['cagri_count']).hist(bins=50)
plt.show()

In [ ]:
corr = df[num_cols].corr()
corr

# Korelasyonların gösterilmesi
sns.set(rc={'figure.figsize': (12, 12)})
sns.heatmap(corr, cmap="RdBu")
plt.show()

In [ ]:
def high_correlated_cols(dataframe, plot=False, corr_th=0.70):
    corr = dataframe.corr()
    cor_matrix = corr.abs()
    upper_triangle_matrix = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))
    drop_list = [col for col in upper_triangle_matrix.columns if any(upper_triangle_matrix[col] > corr_th)]
    if plot:
        import seaborn as sns
        import matplotlib.pyplot as plt
        sns.set(rc={'figure.figsize': (15, 15)})
        sns.heatmap(corr, cmap="RdBu")
        plt.show()
    return drop_list


high_correlated_cols(df, plot=False)

In [ ]:
df = df.drop('OutageID', axis=1)

In [ ]:
df.corr()

In [ ]:
def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]

    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")

    if na_name:
        return na_columns

missing_values_table(df)

This is the NaN value of the target variable(cagri_count) in the test set.

In [ ]:
######################################
# Feature Engineering:
######################################

# 1. Date Transformation:

df.BAŞLAMA_TARİHİ_VE_ZAMANI=pd.to_datetime(df.BAŞLAMA_TARİHİ_VE_ZAMANI,dayfirst=True)
df.SONA_ERME_TARİHİ_VE_ZAMANI=pd.to_datetime(df.SONA_ERME_TARİHİ_VE_ZAMANI,dayfirst=True)


df['baslangic_month']=df.BAŞLAMA_TARİHİ_VE_ZAMANI.dt.month
df['sona_erme_month']=df.SONA_ERME_TARİHİ_VE_ZAMANI.dt.month

df['baslangic_quarter']=df.BAŞLAMA_TARİHİ_VE_ZAMANI.dt.quarter
df['sona_erme_quarter']=df.SONA_ERME_TARİHİ_VE_ZAMANI.dt.quarter

df['baslangic_day']=df.BAŞLAMA_TARİHİ_VE_ZAMANI.dt.day
df['sona_erme_day']=df.SONA_ERME_TARİHİ_VE_ZAMANI.dt.day

df['baslangic_dayofweek']=df.BAŞLAMA_TARİHİ_VE_ZAMANI.dt.dayofweek
df['sona_erme_dayofweek']=df.SONA_ERME_TARİHİ_VE_ZAMANI.dt.dayofweek


df['hour'] = df.BAŞLAMA_TARİHİ_VE_ZAMANI.dt.hour
df['son_hour'] = df.SONA_ERME_TARİHİ_VE_ZAMANI.dt.hour

In [ ]:
def encode_hours(df, column):
    df.loc[(df[column] > 15) & (df[column] < 24), column+"_dilimi"] = "eglence"
    df.loc[(df[column] > 0) & (df[column] < 9), column+"_dilimi"] = "uyku"
    df[column+"_dilimi"] = df[column+"_dilimi"].fillna("is") 
    return df

df = encode_hours(df, "hour")
df = encode_hours(df, "son_hour")

In [ ]:
df['week_of_year_Bas'] = df.BAŞLAMA_TARİHİ_VE_ZAMANI.dt.weekofyear
#df['week_of_year_Son'] = df.SONA_ERME_TARİHİ_VE_ZAMANI.dt.weekofyear

In [ ]:
df['month_difference'] = df['sona_erme_month'] - df['baslangic_month']
df['quarter_difference'] = df['sona_erme_quarter'] - df['baslangic_quarter']
df['day_difference'] = df['sona_erme_day'] - df['baslangic_day']
df['dayofweek_difference'] = df['sona_erme_dayofweek'] - df['baslangic_dayofweek']

In [ ]:
from unidecode import unidecode
izmir_ilce = pd.read_html('https://www.nufusu.com/ilceleri/izmir-ilceleri-nufusu')[0]
manisa_ilce = pd.read_html('http://www.manisa.gov.tr/nufus-ve-dagilim')[0].iloc[:-1,:]
izmir_ilce=izmir_ilce[['İlçe','Toplam Nüfus']].rename(columns={'İlçe':'İLÇE','Toplam Nüfus':'ilce_nufus'})
manisa_ilce=manisa_ilce[['İlçe','İlçe Nüfusu']].rename(columns={'İlçe':'İLÇE','İlçe Nüfusu':'ilce_nufus'})
ilce_nufus=pd.concat([izmir_ilce,manisa_ilce])
ilce_nufus['İLÇE']=ilce_nufus['İLÇE'].apply(unidecode).str.lower()

df['İL']=df['İL'].apply(unidecode).str.lower()
df['İLÇE']=df['İLÇE'].apply(unidecode).str.lower()

df=pd.merge(df,ilce_nufus,on=['İLÇE'],how='left')
df['ilce_nufus'] = df['ilce_nufus'].astype(float)
df['Nufus'] = pd.cut(x = df['ilce_nufus'], bins = [0, 57000, 170000, 348000, 525000], labels = ['Low', 'Moderate', 'High', 'Extreme'])

In [ ]:
#df['Nufus_Kesinti'] = df['ilce_nufus'] * df['KESİNTİ_SÜRESİ']  
#df['Nufus_AG'] =  (df['ilce_nufus'] /df['TOPLAM_KENTSEL_AG']) * df['KESİNTİ_SÜRESİ']
#df['Nufus_OG'] =  (df['ilce_nufus'] /df['TOPLAM_KENTSEL_OG']) * df['KESİNTİ_SÜRESİ']

In [ ]:
# YENI FEATURE AFTER 30.46. 30.46 ilcin bir onceki notebook a /365 ekledim

In [ ]:
#ilce_kesinti_sayisi = df.groupby('İLÇE').agg(ilce_kesinti_sayi = ('KESİNTİ_SÜRESİ', 'count'))
#ilce_kesinti_sayisi.head()

#df=pd.merge(df,ilce_kesinti_sayisi,on=['İLÇE'],how='left')

In [ ]:
#df['ilce_kesinti_sayi'].head()

In [ ]:
## YENI FEATURE
#ilce_kesinti_sure_top = df.groupby('İLÇE').agg(ilce_kesinti_sure_toplam = ('KESİNTİ_SÜRESİ', 'sum'))
#ilce_kesinti_sure_top.head()


#df=pd.merge(df,ilce_kesinti_sure_top,on=['İLÇE'],how='left')

#df['Oran_kesinti_sure_ilce'] = df['KESİNTİ_SÜRESİ']/(df['ilce_kesinti_sure_toplam']/df['ilce_kesinti_sayi'])

#df['Ortalama_kesinti_sure_ilce'] = df['ilce_kesinti_sure_toplam']/df['ilce_kesinti_sayi']

In [ ]:
# Yeni Feature
#df['a']= df['ilce_kesinti_sayi']/ df['KENTSEL_AG']

In [ ]:
mylist = [10,11,12,1,2]
df['Fideri'] = df.apply(lambda x: 1 if  'Fideri' in x['ŞEBEKE_UNSURU'] and x['baslangic_month'] in mylist else 0 ,axis=1)

In [ ]:
# Saatlerin gruplandirilmasi
df['HOURS_NEW'] = pd.cut(x = df['hour'], bins = [0, 6, 12, 18, 24], labels = ['Early', 'Morning', 'Midday', 'Night'])

In [ ]:
# Nufus az-orta-cok-extreme diye gruplandirilmisti. Bunu kesinti saati erken-sabah-gunortasi-gece olarak birlestirdim.
df['NUFUS_HOURS_NEW'] = df['Nufus'].astype(str) + df['HOURS_NEW'].astype(str)

In [ ]:
# Kesinti suresi negatif degerliydi. Pozitife cevirdim.
df["KESİNTİ_SÜRESİ"] = df["KESİNTİ_SÜRESİ"].abs()

In [ ]:
# Saat cinsinden ifade edilen kesinti suresinden KENTTEKI ALCAK gerilimli hatlardaki gunluk ortalama kesinti suresini cikardim.
df['NEW_AG'] = df['KESİNTİ_SÜRESİ']-df['TOPLAM_KENTSEL_AG']/365
# Saat cinsinden ifade edilen kesinti suresinden KENTTEKI ORTA gerilimli hatlardaki gunluk ortalama kesinti suresini cikardim.
df['NEW_OG'] = df['KESİNTİ_SÜRESİ']-df['TOPLAM_KENTSEL_OG']/365
# Saat cinsinden ifade edilen kesinti suresinden KIRSALDAKI ORTA gerilimli hatlardaki gunluk ortalama kesinti suresini cikardim.
df['NEW_K_OG'] = df['KESİNTİ_SÜRESİ']-df['TOPLAM_KIRSAL_OG']/365
# Saat cinsinden ifade edilen kesinti suresinden KIRSALDAKI ALCAK gerilimli hatlardaki gunluk ortalama kesinti suresini cikardim.
df['NEW_K_AG'] = df['KESİNTİ_SÜRESİ']-df['TOPLAM_KIRSAL_AG']/365
# Saat cinsinden ifade edilen kesinti suresinden KENTALTI ALCAK gerilimli hatlardaki gunluk ortalama kesinti suresini cikardim.
df['NEW_AG_2'] = df['KESİNTİ_SÜRESİ']-df['TOPLAM_KENTALTI_AG']/365
# Saat cinsinden ifade edilen kesinti suresinden KENTALTI ORTA gerilimli hatlardaki gunluk ortalama kesinti suresini cikardim.
df['NEW_OG_2'] = df['KESİNTİ_SÜRESİ']-df['TOPLAM_KENTALTI_OG']/365
# KENTTEKI TOPLAM ALCAK VE ORTA GERILIM HAT SAYISINI, KESINTI SURESI ILE CARPTIM.
df['NEW_KENTSEL'] = (df['KENTSEL_AG'] + df['KENTSEL_OG'])*df['KESİNTİ_SÜRESİ']
# KENTALTINDAKI TOPLAM ALCAK VE ORTA GERILIM HAT SAYISINI, KESINTI SURESI ILE CARPTIM.
df['NEW_KENTALTI'] = (df['KENTALTI_AG'] + df['KENTALTI_OG'])*df['KESİNTİ_SÜRESİ']
# KIRSALDAKI TOPLAM ALCAK VE ORTA GERILIM HAT SAYISINI, KESINTI SURESI ILE CARPTIM.
df['NEW_KIRSAL'] = (df['KIRSAL_AG'] + df['KIRSAL_OG'])*df['KESİNTİ_SÜRESİ']
# ALCAK GERILIMLI HATLARDAKI YIL ICINDE YASANAN TOPLAM KESINTI SAYISINI BULUP, 365 E BOLDUM. ORTALAMA ALCAK GERILIMLI HATTA CIKAN KESINTI SURESINI VERDI.
df['TOPLAM_AG'] = (df['TOPLAM_KENTSEL_AG']+ df['TOPLAM_KIRSAL_AG']+df['TOPLAM_KENTALTI_AG'])/365
# ORTA GERILIMLI HATLARDAKI YIL ICINDE YASANAN TOPLAM KESINTI SAYISINI BULUP, 365 E BOLDUM. ORTALAMA ORTA GERILIMLI HATTA CIKAN KESINTI SURESINI VERDI.
df['TOPLAM_OG'] = (df['TOPLAM_KENTSEL_OG']+df['TOPLAM_KIRSAL_OG']+df['TOPLAM_KENTALTI_OG'])/365

df['TOPLAM_HAT_SAYISI'] = df['KENTSEL_AG']+df['KENTSEL_OG']+df['KENTALTI_AG']+df['KENTALTI_OG']+df['KIRSAL_AG']+df['KIRSAL_OG']

In [ ]:
#df['TOPLAM_HAT_SAYISI_NUFUS'] = df['ilce_nufus'] / df['TOPLAM_HAT_SAYISI']

In [ ]:
# IZMIR ICIN SEHRIN ICINDEKI ILCELERI YAKIN, SEHRE UZAK ILCELERE UZAK YAPISTIRDIM.
df['UZAKLIK'] = df['İLÇE'].apply(lambda x: 'UZAK' if x not in ['BORNOVA', 'KONAK','BUCA', 'KARABAĞLAR', 'KARŞIYAKA', 'BAYRAKLI', 'GAZIEMIR', 'NARLIDERE', 'BALÇOVA'] else 'YAKIN' )
# BU UZAKLIK YAKINLIGI SAYI ILE IFADE ETTIM.
df['UZAKLIK_SAYI']= df['UZAKLIK'].apply(lambda x: 2 if 'UZAK' else 1)
df['UZAK_HOUR'] = df['UZAKLIK'].apply(lambda x: 2 if 'UZAK' else 1)*df['hour']

In [ ]:
#ILCE NUFUSUNU NEW KENTSELE BOLDUM DE BUNU NIYE YAPTIM??
df['NUFUS_KENTSEL_NEW'] = df['ilce_nufus']/(df['NEW_KENTSEL'])

In [ ]:
# ILCE NUFUSUNU TOPLAM ALCAK VE ORTA GERILIMLI HAT SAYISINA BOLDUM
df['NUFUS_KENTSEL_AGOG'] = df['ilce_nufus']/(df['KENTSEL_AG'] + df['KENTSEL_OG'])
df['NUFUS_KENTALTI_AGOG'] = df['ilce_nufus']/(df['KENTALTI_AG'] + df['KENTALTI_OG'])
df['NUFUS_KIRSAL_AGOG'] = df['ilce_nufus']/(df['KIRSAL_AG'] + df['KIRSAL_OG'])

In [ ]:
# ILCE NUFUSNU KENTTEKI ALCAK GERLIMLI HAT SAYISINA BOLUP , KENTTEKI ALCAK GERILIMLI HAT SAYISINDA CIKAN TOPLAM KESINTI MIKTARI ILE CARPTIM.
df['AG_TOPLAM_KENTSEL'] = (df['ilce_nufus'] / df['KENTSEL_AG'])*df['TOPLAM_KENTSEL_AG']

In [ ]:
# ILCE NUFUSUNU KENTTEKI ORTA VE ALCAK GERILIMLI HATLARDA YASANAN TOPLEM KESINTI SAYISINA BOLDUM. KISI BASI KESINTI SAYISI GIBI BIR SEY CIKTI, 
# CIKAN SONUCU ILCENIN KENTE UZAKLIGI ILE CARPTIM.
df['NUFUS_UZAKLIK'] = df['ilce_nufus']/(df['TOPLAM_AG']+df['TOPLAM_OG'])*df['UZAK_HOUR'].astype(int)

In [ ]:
# KESINTI SURESINI KATEGORILERE AYIRDIM.
df['KESINTI_NEW'] = pd.cut(x = df['KESİNTİ_SÜRESİ'], bins = [-1,3,10,30,100000],
                               labels = ['Low','Normal', 'Extreme', 'XXXextreme'])

In [ ]:
# KATEGORILERE AYRILAN KESINTI SURESINI SAAT KATEGORILERI ILE BIRLESTIRDIM.
df['KESINTI_HOUR'] = df['KESINTI_NEW'].astype(str) + df['HOURS_NEW'].astype(str)

In [ ]:
# TR ile baslayan sebek unsuru kodlarini aldim.
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.startswith('TR', na=False),'SEBEKE_STARTSWITH'] = 'TR'

In [ ]:
# DM ile baslayan sebeke unsuru kodlarini aldim
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.startswith('DM', na=False),'SEBEKE_STARTSWITH'] = 'DM'

In [ ]:
# TRAFO iceren sebeke kodlarini aldim.
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('DIREK TIPI TRAFO HUCRESI'),'SEBEKE_TRAFO'] = 'TRAFO'

In [ ]:
# Petrol iceren sebeke unsuru kodlarini aldim.
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('PETROL'),'SEBEKE_PETROL'] = 'PETROL'

In [ ]:
# TARIM ve turevi iceren sebeke kodlarini aldim.
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('TARIM' or 'TARIMSAL' or 'SULAMA' or'TOPRAK'or 'SLM'),'SEBEKE_TARIM'] = 'TARIM'

In [ ]:
# Koy icerenler
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('KÖY'),'SEBEKE_KOY'] = 'KOY'
# OZEL icerenler
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('ÖZEL'),'SEBEKE_OZEL'] = 'ÖZEL'
# Telco ismi iceren sebek kodlarini aldim
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('TURKCELL' or 'TÜRK TELEKOM' or 'VODAFONE'),'SEBEKE_TELCO'] = 'TELCO'
# Industry iceren sebeke kodlarini aldim.
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('FABRİKA' or 'SANAYİ' or 'ORGANİZE'),'SEBEKE_INDUSTRY'] = 'INDUSTRY'

In [ ]:
# olusturulan degiskenler icin kalan kisimlar nan olacagi icin nan'leri other yaptim.
df['SEBEKE_STARTSWITH'].fillna('Other',inplace = True)
df['SEBEKE_TRAFO'].fillna('Other',inplace = True)
df['SEBEKE_PETROL'].fillna('Other',inplace = True)
df['SEBEKE_TARIM'].fillna('Other',inplace = True)
df['SEBEKE_KOY'].fillna('Other',inplace = True)
df['SEBEKE_OZEL'].fillna('Other',inplace = True)
df['SEBEKE_TELCO'].fillna('Other',inplace = True)
df['SEBEKE_INDUSTRY'].fillna('Other',inplace = True)
#df['SEBEKE_STARTSWITH'].fillna('Other',inplace = True)

In [ ]:
#M05_72256924 
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M001'),'AMH KODU'] = 5.499/2.030
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M002'),'AMH KODU'] = 6.923/2.334
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M003'),'AMH KODU'] = 6.853/2.086
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M004'),'AMH KODU'] = 5.499/2.036
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M005'),'AMH KODU'] = 6.496/2.532
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M006'),'AMH KODU'] = 7.133/2.187
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M007'),'AMH KODU'] = 8.324/2.110
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M008'),'AMH KODU'] = 10.173/2.181
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M009'),'AMH KODU'] = 8.097/2.848
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M010'),'AMH KODU'] = 9.288/2.706
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M011'),'AMH KODU'] = 7.560/2.192
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M012'),'AMH KODU'] = 12.766/2.483
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M013'),'AMH KODU'] = 8.821/2.281
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('M014'),'AMH KODU'] = 9.343/3.128

df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('A00001'),'AMH KODU'] = 20.662/0.855
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('A00002'),'AMH KODU'] = 32.173/1.010
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('A00003'),'AMH KODU'] = 42.253/1.055
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('A00004'),'AMH KODU'] = 30.356/0.794
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('A00005'),'AMH KODU'] = 26.317/1.069
df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('A00006'),'AMH KODU'] = 47.670/0.716

df.loc[df['ŞEBEKE_UNSURU_KODU'].str.contains('H0'),'AMH KODU'] = 7.444/2.705

df['AMH KODU'] = df['AMH KODU'].astype(float)*df['hour']

In [ ]:
# Kademe gereksiz.
df = df.drop('KADEME',axis=1)

In [ ]:
# UZAK-YAKIN I KALABALIK-AZ KALABALIK OLARAK BIRLESTIRDIK.
df['UZAKLIKvNUFUS'] = df['UZAKLIK'].astype(str)+df['Nufus'].astype(str)

In [ ]:
# KESINTI SURESI ile kesinti SAATINI VE uzakligi carptim. burada saati iki kere carptim, ama bi kere carparsam rmse yuksek cikiyodu :D
df['KESINTI_HOUR'] = df['KESİNTİ_SÜRESİ']*df['hour']*df['UZAK_HOUR']

In [ ]:
# ustteki carpima toplam alcak gerilimdeki kesinti suresini ekledim.
#df['KESINTI_HOUR_2'] = df['KESİNTİ_SÜRESİ']*df['hour']*df['UZAK_HOUR']*df['TOPLAM_KENTSEL_AG']

In [ ]:
# Kategorilere ayrilan kesinti suresi ile uzaklik birlesti
df['KESINTIvUZAKLIK']= df['UZAKLIK'].astype(str)+df['KESINTI_NEW'].astype(str)

In [ ]:
#df['UZAK_HOUR'] = df['UZAKLIK'].astype(str) + df['HOURS_NEW'].astype(str)

In [ ]:
#df['Feature1']= df['UZAKLIK']+'_'+df['baslangic_month']

In [ ]:
df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].value_counts()

In [ ]:
print(col, ":", len(df['İLÇE'].value_counts()))
print(pd.DataFrame({"COUNT": df['İLÇE'].value_counts(),
                    "RATIO": df['İLÇE'].value_counts() / len(df),
                    "TARGET_MEAN": df.groupby('İLÇE')['cagri_count'].mean()}).sort_values('TARGET_MEAN',ascending = False), end="\n\n\n")

In [ ]:
# Burada ilceleri target meanlerine gore grupladim.
df['İLÇE'] = df['İLÇE'].str.upper()
df.loc[df['İLÇE'].str.contains("BAYRAKLI" or 'CIGLI' or 'KARABAGLAR'), "ilce_grup"] = 'BCK'
df.loc[df['İLÇE'].str.contains("GAZİEMİR" or 'KONAK' or 'NARLIDERE'), "ilce_grup"] = 'GKN'
df.loc[df['İLÇE'].str.contains("BORNOVA" or 'KARSIYAKA'), "ilce_grup"] = 'BK'
df.loc[df['İLÇE'].str.contains("FOCA" or 'MENEMEN' or 'ALİAGA' or 'BUCA' or 'YUNUSEMRE' or 'SEHZADELER' or 'URLA' or 'SOMA'),"ilce_grup"] = "FMABYS"
df.loc[df['İLÇE'].str.contains("GUZELBAHCE" or 'TORBALI' or 'BERGAMA' or 'BALCOVA' or 'TURGUTLU' or 'KEMALPASA' or 'MENDERES'), "ilce_grup"] = 'GTBBTKM'
df.loc[df['İLÇE'].str.contains("DIKILI" or 'SARUHANLI' or 'SEFERIHISAR' or 'ALASEHIR' or 'ÇESME' or 'KIRKAGAC' or 'AKHISAR' or 'ODEMIS' or 'KARABURUN' or 'SELCUK' or 'TIRE'), "ilce_grup"] = 'cok'
df.loc[df['İLÇE'].str.contains("SALIHLI" or 'BAYINDIR' or 'AHMETLI' or 'SELENDI'), "ilce_grup"] = 'low2'
df.loc[df['İLÇE'].str.contains("KULA" or 'KIRAZ' or 'GORDES'or 'KINIK' or 'KOPRUBASI' or 'SARIGOL' or 'DEMIRCI' or 'GOLMARMARA' or'BEYDAG '), "ilce_grup"] = "low1"

In [ ]:
df['ilce_grup'].value_counts()

In [ ]:
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('OG Direk Yıkılması'), 'Yuksek Sureli Arizalar'] =  414.835
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('OG Trafo Arızası' or 'AG Pano Arızası' or 'AG Yeraltı Kablo Arızası' or 'OG İletken Kopması'), 'Yuksek Sureli Arizalar'] = 283
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('AG Direk Kırılması'or 'AG Branşman Yeraltı Kablo Arızası'), 'Yuksek Sureli Arizalar']=240
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('Hırsızlık' or 'OG Travers Arızası' or 'Şebeke Bakım Çalışması'), 'Yuksek Sureli Arizalar'] =230
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('OG Yeraltı Kablo Arızası ' or 'AG Hatta Yabancı Cisim ' or 'OG İzolatör Arızası '), 'Yuksek Sureli Arizalar' ] =215

df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('AG Tel Kopuğu' or 'Üçüncü Şahısların Vermiş Olduğu Hasarlar' or 'AG Direk Değişimi' or 'AG Pano Faz Sigorta Atığı' or 'AG Atlama Kopuğu  '), 'Yuksek Sureli Arizalar'] =195

df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('OG Kablo Başlığı Arızası' or 'AG Nötr İletken Kopması' or 'AG Kablo Başlığı Arızası'or 'AG Klemens Arızası' or 'AG Sehim Bozukluğu'), 'Yuksek Sureli Arizalar'] = 176
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('AG Havai Branşman Arızası' or 'AG Box Arızası' or 'OG Atlama Arızası' or 'Yangın' or 'AG İzolatör Arızası' or 'OG Atlama(Jumper) Arızası'), 'Yuksek Sureli Arizalar'] =165
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('AG Pano Kol Sigorta Atığı' or 'OG Sigorta Atması' or 'AG Sigorta Atığı' or 'NH Altlık Arızası'), 'Yuksek Sureli Arizalar'] =155
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('AG Box / Sdk Giriş Sigorta Atığı' or 'OG Klemens Arızası' or 'OG Ayırıcı Arızası' or'AG Termik Açması'), 'Yuksek Sureli Arizalar'] =143
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('AG Box / Sdk Abone Çıkış Faz Sigorta Atığı' or 'AG Box / Sdk Abone Çıkış Sigorta Atığı' or 'Müteahhit Çalışması' or 'OG Kesici Arızası' or 'Trafo Bakım Çalışması' or 'AG Box / Sdk Giriş Faz Sigorta Atığı'), 'Yuksek Sureli Arizalar']=134
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('AG Direkten Kesme Açma' or 'AG Box / Sdk Abone Çıkış Sigorta Atığı'), 'Yuksek Sureli Arizalar'] =113
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('Fiziki İrtibat' or 'OG Hatta Ağaç Kesimi'), 'Yuksek Sureli Arizalar'] =100
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('Ağaç Kesimi' or 'Yük Aktarımı ' or 'Yangın Sonucu Şebeke Hasarı' or 'Yabani Hayvan Teması' or 'AG Kademe Ayarı' or 'EİH Arızası' or 'TEİAŞ Trafo Arızası'), 'Yuksek Sureli Arizalar'] =72
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('OG Fider Açması' or 'OG Trafo Kademe Ayarı' or 'OG Fider Açması'), 'Yuksek Sureli Arizalar'] =54.218
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('Manevra' or 'AG Voltaj Düşüklüğü'), 'Yuksek Sureli Arizalar'] =18
df['Yuksek Sureli Arizalar']=df['Yuksek Sureli Arizalar'].fillna(140)
df['Yuksek Sureli Arizalar'] = df['Yuksek Sureli Arizalar'].astype(int)

In [ ]:
df['Yuksek Sureli Arizalar'].value_counts()
#df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].unique()

In [ ]:
df['a'] = df['Yuksek Sureli Arizalar']*df['hour']*df['UZAKLIK_SAYI'].astype(int)

In [ ]:
print(col, ":", len(df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].value_counts()))
print(pd.DataFrame({"COUNT": df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].value_counts(),
                    "RATIO": df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].value_counts() / len(df),
                    "TARGET_MEAN": df.groupby('KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA' )['KESİNTİ_SÜRESİ'].mean()}).sort_values('TARGET_MEAN',ascending = False), end="\n\n\n")

In [ ]:
print(col, ":", len(df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].value_counts()))
print(pd.DataFrame({"COUNT": df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].value_counts(),
                    "RATIO": df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].value_counts() / len(df),
                    "TARGET_MEAN": df.groupby('KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA' )['KESİNTİ_SÜRESİ'].mean()}).sort_values('TARGET_MEAN',ascending = False), end="\n\n\n")

In [ ]:
# Kesinti nedeninden Ag ve Og yi cikarttim yeni degisken elde ettim.
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('AG'),'KESINTI_NEDENI'] = 'AG'
df.loc[df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].str.contains('OG'),'KESINTI_NEDENI'] = 'OG'

In [ ]:
df['KESINTI_NEDENI']=df['KESINTI_NEDENI'].fillna('Other')

In [ ]:
# Kesinti nedeni ile  nufus yogunlugunu string olarak grupladim,
df['NufusvKesintiNeden'] = df['KESINTI_NEDENI'].astype(str) + df['Nufus'].astype(str)

In [ ]:
df['KESINTI_NEDENI'].value_counts()

In [ ]:
# Kesinti nedenine iliskin aciklamalari target mean lerine gore grupladim.
df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["Şebeke Bakım Çalışması", "EİH Arızası", "Yangın Sonucu Şebeke Hasarı", 
                                                                'Müteahhit Çalışması', 'OG Trafo Kademe Ayarı', 'Trafo Bakım Çalışması',
                                                                'OG Travers Arızası', 'TEİAŞ Hat Bakım Çalışması']), 1 , df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])
df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["AG Direk Kırılması", "AG Voltaj Düşüklüğü" , "AG Box / Sdk Abone Çıkış Faz Sigorta Atığı",
                                                                                              "AG Hatta Ağaç Kesimi", "AG Havai Branşman Arızası",  "AG Direk Değişimi" ,'AG Travers Arızası']), 2 , df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])

df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["AG Kademe Ayarı", "AG Branşman Yeraltı Kablo Arızası" , "AG Box / Sdk Abone Çıkış Sigorta Atığı",
                                                                                              "OG Kademe Ayarı", "AG Nötr İletken Kopması",  "AG Sehim Bozukluğu" , 'AG Box Arızası', 'AG İzolatör Arızası']), 3 , df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])

df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["AG Direkten Kesme Açma", "AG Voltaj Düşüklüğü" , "AG Box / Sdk Abone Çıkış Faz Sigorta Atığı",
                                                                                              "AG Hatta Ağaç Kesimi", "AG Havai Branşman Arızası",  "AG Direk Değişimi" ,'AG Travers Arızası']),  4 , df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])

df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["AG Box / Sdk Giriş Faz Sigorta Atığı", "Manevra" , "AG Kablo Başlığı Arızası",
                                                                                              "OG Sigorta Atması" , 'AG Tel Kopuğu']),  5, df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])
df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["AG Atlama Kopuğu", "AG Pano Kol Sigorta Atığı" , "OG Ayırıcı Arızası",
                                                                                              "OG Hatta Ağaç Kesimi"]), 6 , df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])
df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["AG Pano Faz Sigorta Atığı", "Yük Aktarımı" , "Hırsızlık",
                                                                                              "AG Box / Sdk Giriş Sigorta Atığı", "AG Termik Açması" ]),  7 , df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])
                                                                                              
                                                                                              

In [ ]:
df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["TEİAŞ Trafo Arızası", "Yabani Hayvan Teması" ]), 15, df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])

df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["AG Pano Arızası", "OG Yeraltı Kablo Arızası" ]), 14, df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])

df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["OG Kesici Arızası", "OG Hatta Yabancı Cisim" ]), 13, df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])

df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["Üçüncü Şahısların Vermiş Olduğu Hasarlar", "OG Atlama(Jumper) Arızası", "OG Trafo Arızası", "OG Parafudr Patlaması"]), 12, df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])

df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["Yangın", "OG İletken Kopması", "OG Klemens Arızası",
                                                                                              "OG İzolatör Arızası", "AG Yeraltı Kablo Arızası", "OG Kablo Başlığı Arızası", "AG Sigorta Atığı"]), 11, df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])

df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["NH Altlık Arızası", "OG Fider Açması", "OG Direk Yıkılması"]), 10, df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])

df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin(["AG Yük Ayırıcı Arızası", "Fiziki İrtibat"]), 9, df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])

df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = np.where(df.KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA.isin([ "Ağaç Kesimi", "AG Klemens Arızası" , "OG Atlama Arızası", "AG Hatta Yabancı Cisim", "Plansız Kesinti / Müdahale"]), 8, df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"])

In [ ]:
df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"] = df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"].astype(int)

In [ ]:
df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"].unique()

In [ ]:
df['KES'] =df['ilce_nufus']*df['Yuksek Sureli Arizalar']*df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"]*df['hour']*df['UZAKLIK_SAYI'].astype(int)/df['KENTSEL_AG']

In [ ]:
df.corr()['cagri_count'].sort_values()

In [ ]:
#df['KENTSEL_AG'].describe()
#df['KENTSEL_SINIF'] = pd.cut(x = df['KENTSEL_AG'], bins = [-1, 10, 100, 4000, 15000,25000,45000,55000,75000], labels=[1 , 2, 3, 4,5,6,7,8])
#df['KENTSEL_SINIF']=df['KENTSEL_SINIF'].astype(int)

In [ ]:
#df['ARIZA_SURE_NUFUS'] = df['KESINTI_NEW'].astype(str)+df["KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA"].astype(str)+df['Nufus'].astype(str)

In [ ]:
df['ARIZA_SURE_NUFUSKAG'] = df['ilce_nufus']/(df['KENTSEL_AG']*df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].astype(int))
df['ARIZA_SURE2_NUFUSKOG'] = df['ilce_nufus']/(df['KENTSEL_OG']*df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].astype(int))                                  
#df['ARIZA_SURE3_NUFUSKIAG'] = df['ilce_nufus']/df['KIRSAL_AG']*df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].astype(int)
#df['ARIZA_SURE4_NUFUSKIOG'] = df['ilce_nufus']/df['KIRSAL_OG']*df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'].astype(int)

In [ ]:
print(col, ":", len(df['KAYNAĞA_GÖRE'].value_counts()))
print(pd.DataFrame({"COUNT": df['KAYNAĞA_GÖRE'].value_counts(),
                    "RATIO": df['KAYNAĞA_GÖRE'].value_counts() / len(df),
                    "TARGET_MEAN": df.groupby('KAYNAĞA_GÖRE' )['KESİNTİ_SÜRESİ'].mean()}).sort_values('TARGET_MEAN',ascending = False), end="\n\n\n")

In [ ]:
df.loc[df['KAYNAĞA_GÖRE'].str.contains('Dağıtım-AG'),'KAYNAGA_GORE_NUMARA'] = 2.75
df.loc[df['KAYNAĞA_GÖRE'].str.contains('Dağıtım-OG'),'KAYNAGA_GORE_NUMARA'] = 1.420
df.loc[df['KAYNAĞA_GÖRE'].str.contains('İletim'),'KAYNAGA_GORE_NUMARA'] = 0.792

In [ ]:
df['KAYNAGA_GORE_NUMARA']= df['KAYNAGA_GORE_NUMARA'].astype(int)

In [ ]:
######################################
# RARE
######################################
cat_cols, cat_but_car, num_cols = grab_col_names(df)

# Kategorik kolonların dağılımının incelenmesi
def rare_analyser(dataframe, target, cat_cols):
    for col in cat_cols:
        print(col, ":", len(dataframe[col].value_counts()))
        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                            "RATIO": dataframe[col].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(col)[target].mean()}), end="\n\n\n")

rare_analyser(df, "cagri_count", cat_cols)

In [ ]:
#ŞEBEKE_UNSURU : 8
#                           COUNT  RATIO  TARGET_MEAN
#AG Fideri                  29381  0.241        0.949
#Abone Tesisi               40592  0.332        0.204
#DM                         12035  0.099        2.218
###Dağıtım Transformatörü      9589  0.079        1.330
###KÖK                         9438  0.077        1.037
#OG Fideri                  13970  0.114        1.762
###Saha Dağıtım Kutusu (SDK)   4409  0.036        1.220
#TM Fideri                   2729  0.022        9.496

df["ŞEBEKE_UNSURU"] = np.where(df.ŞEBEKE_UNSURU.isin(["KÖK", "Saha Dağıtım Kutusu (SDK)", "Dağıtım Transformatörü"]), "KOKvSAHADAGITIMvDAGTR", df["ŞEBEKE_UNSURU"])

#SEBEBE_GÖRE : 4
#G                     COUNT  RATIO  TARGET_MEAN
#0                        2  0.000        0.000
#Dışsal                2421  0.020        5.912
#Güvenlik               150  0.001        0.748
#Şebeke işletmecisi  119570  0.979        1.048

df["SEBEBE_GÖRE"] = np.where(df.SEBEBE_GÖRE.isin(["0","GUVENLIK"]), "0veGuvenlik", df["SEBEBE_GÖRE"])


In [ ]:
print(col, ":", len(df['İLÇE'].value_counts()))
print(pd.DataFrame({#"COUNT": df['hour'].value_counts(),
                    #"RATIO": df['hour'].value_counts() / len(df),
                    "KENTSEL AG": df[(df['İLÇE'] == 'cigli') & (df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'] == 'Üçüncü Şahısların Vermiş Olduğu Hasarlar')].groupby(['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA', 'İLÇE','KESİNTİ_SÜRESİ', 'hour', 'ŞEBEKE_UNSURU_KODU', 'baslangic_month','AMH KODU'])['KENTSEL_AG'].mean(),
                    "TARGET_MEAN": df[(df['İLÇE'] == 'cigli') & (df['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA'] == 'Üçüncü Şahısların Vermiş Olduğu Hasarlar')].groupby(['KESİNTİ_NEDENİNE_İLİŞKİN_AÇIKLAMA', 'İLÇE', 'KESİNTİ_SÜRESİ', 'hour', 'ŞEBEKE_UNSURU_KODU' ,'baslangic_month', 'AMH KODU'])['cagri_count'].mean()}).sort_values(['TARGET_MEAN'],ascending = False), end="\n\n\n")



In [ ]:
df = df.drop('BAŞLAMA_TARİHİ_VE_ZAMANI', axis =1)
df = df.drop('SONA_ERME_TARİHİ_VE_ZAMANI', axis =1)

In [ ]:
df.drop('ŞEBEKE_UNSURU_KODU',axis=1,inplace= True)

In [ ]:
##################
# GÖREV 4 : Label Encoding & One-Hot Encoding işlemlerinin uygulanması
##################


cat_cols, cat_but_car, num_cols = grab_col_names(df)

def label_encoder(dataframe, binary_col):
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe
binary_cols = [col for col in df.columns if df[col].dtypes == "O"
               and len(df[col].unique()) == 2 and col != 'ind']

for col in binary_cols:
    label_encoder(df, col)

In [ ]:
cat_cols, cat_but_car, num_cols = grab_col_names(df)
def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

cat_cols = [col for col in cat_cols if col not in 'ind']


df = one_hot_encoder(df, cat_cols, drop_first=True)

In [ ]:
df.head()

In [ ]:
###############################
# MODELLEME
##############################


test, train = df[df["ind"].eq("test")], df[df["ind"].eq("train")]

#
y = (train['cagri_count'])
X = train.drop(["KESİNTİ_NO", "cagri_count", "ind", 'İLÇE'], axis=1)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=17)

In [ ]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV

In [ ]:
models = [('LR', LinearRegression()),
          ("Ridge", Ridge()),
          ("Lasso", Lasso()),
          ("ElasticNet", ElasticNet()),
          ('KNN', KNeighborsRegressor()),
          ('CART', DecisionTreeRegressor()),
          ('RF', RandomForestRegressor()),
          ('SVR', SVR()),
          ('GBM', GradientBoostingRegressor()),
          ("XGBoost", XGBRegressor(objective='reg:squarederror')),
          ("LightGBM", LGBMRegressor())]
          # ("CatBoost", CatBoostRegressor(verbose=False))]

In [ ]:
#for name, regressor in models:
    #rmse = np.mean(np.sqrt(-cross_val_score(regressor, X, y, cv=5, scoring="neg_mean_squared_error")))
    #print(f"RMSE: {round(rmse, 4)} ({name}) ")


In [ ]:
"""from sklearn.ensemble import VotingRegressor, StackingRegressor

estimators = [('lgbm', LGBMRegressor(verbose=-1)), 
             ('catboost', CatBoostRegressor(verbose=False)),
             ('xgboost', XGBRegressor())]

model = VotingRegressor(estimators=estimators)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
"""

In [ ]:
lgbm_model = LGBMRegressor(random_state=46)
rmse = np.mean(np.sqrt(-cross_val_score(lgbm_model,
                                        X, y, cv=5, scoring="neg_mean_squared_error")))
rmse #23.93

In [ ]:
lgbm_params = {"learning_rate": [0.01, 0.03],
               "n_estimators": [500,1000],
               'colsample_bytree': [0.3,0.7]
             }

lgbm_gs_best = GridSearchCV(lgbm_model,
                            lgbm_params,
                            cv=10,
                            n_jobs=-1,
                            verbose=True).fit(X, y)

In [ ]:
lgbm_gs_best.best_params_

In [ ]:
lgbm_final = lgbm_model.set_params(**lgbm_gs_best.best_params_, random_state = 17).fit(X,y)

In [ ]:
rmse = np.mean(np.sqrt(-cross_val_score(lgbm_final, X, y, cv=5, scoring="neg_mean_squared_error")))

In [ ]:
rmse 
#23.61

In [ ]:
def plot_importance(model, features, num=len(X), save=False):
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features.columns})
    plt.figure(figsize=(10, 30))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')

plot_importance(lgbm_final, X)

In [ ]:
y = test['cagri_count']
X = test.drop(["KESİNTİ_NO", "cagri_count","ind","İLÇE"], axis=1)
y_pred2 = lgbm_final.predict(X)

In [ ]:
y_pred2[y_pred2<0] = 0
y_pred2= [float(round(pred)) for pred in y_pred2]

In [ ]:
my_submission = pd.DataFrame({'KESİNTİ_NO': test.KESİNTİ_NO, 'cagri_count': y_pred2})
# you could use any filename. We choose submission here
my_submission.to_csv('submission502.csv', index=False)